In [1]:
import networkx as nx


import os
import random
from collections import defaultdict

import gensim
import networkx as nx
import numpy as np
import pkg_resources
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from mumin import MuminDataset
import dgl
import torch



import random
from tqdm import tqdm

Using backend: pytorch


In [2]:
def parallel_generate_walks(d_graph: dict, global_walk_length: int, num_walks: int, cpu_num: int,
                            sampling_strategy: dict = None, num_walks_key: str = None, walk_length_key: str = None,
                            neighbors_key: str = None, probabilities_key: str = None, first_travel_key: str = None,
                            quiet: bool = False) -> list:
    """
    Generates the random walks which will be used as the skip-gram input.

    :return: List of walks. Each walk is a list of nodes.
    """

    walks = list()

    if not quiet:
        pbar = tqdm(total=num_walks, desc='Generating walks (CPU: {})'.format(cpu_num))

    for n_walk in range(num_walks):

        # Update progress bar
        if not quiet:
            pbar.update(1)

        # Shuffle the nodes
        shuffled_nodes = list(d_graph.keys())
        random.shuffle(shuffled_nodes)

        # Start a random walk from every node
        for source in shuffled_nodes:

            # Skip nodes with specific num_walks
            if source in sampling_strategy and \
                    num_walks_key in sampling_strategy[source] and \
                    sampling_strategy[source][num_walks_key] <= n_walk:
                continue

            # Start walk
            walk = [source]

            # Calculate walk length
            if source in sampling_strategy:
                walk_length = sampling_strategy[source].get(walk_length_key, global_walk_length)
            else:
                walk_length = global_walk_length

            # Perform walk
            while len(walk) < walk_length:

                walk_options = d_graph[walk[-1]].get(neighbors_key, None)

                # Skip dead end nodes
                if not walk_options:
                    break

                if len(walk) == 1:  # For the first step
                    probabilities = d_graph[walk[-1]][first_travel_key]
                    walk_to = random.choices(walk_options, weights=probabilities)[0]
                else:
                    probabilities = d_graph[walk[-1]][probabilities_key][walk[-2]]
                    walk_to = random.choices(walk_options, weights=probabilities)[0]

                walk.append(walk_to)

            walk = list(map(str, walk))  # Convert all to strings

            walks.append(walk)

    if not quiet:
        pbar.close()

    return walks


class Node2Vec:
    FIRST_TRAVEL_KEY = 'first_travel_key'
    PROBABILITIES_KEY = 'probabilities'
    NEIGHBORS_KEY = 'neighbors'
    WEIGHT_KEY = 'weight'
    NUM_WALKS_KEY = 'num_walks'
    WALK_LENGTH_KEY = 'walk_length'
    P_KEY = 'p'
    Q_KEY = 'q'

    def __init__(self, graph: nx.Graph, dimensions: int = 128, walk_length: int = 80, num_walks: int = 10, p: float = 1,
                 q: float = 1, weight_key: str = 'weight', workers: int = 1, sampling_strategy: dict = None,
                 quiet: bool = False, temp_folder: str = None, seed: int = None):
        """
        Initiates the Node2Vec object, precomputes walking probabilities and generates the walks.

        :param graph: Input graph
        :param dimensions: Embedding dimensions (default: 128)
        :param walk_length: Number of nodes in each walk (default: 80)
        :param num_walks: Number of walks per node (default: 10)
        :param p: Return hyper parameter (default: 1)
        :param q: Inout parameter (default: 1)
        :param weight_key: On weighted graphs, this is the key for the weight attribute (default: 'weight')
        :param workers: Number of workers for parallel execution (default: 1)
        :param sampling_strategy: Node specific sampling strategies, supports setting node specific 'q', 'p', 'num_walks' and 'walk_length'.
        :param seed: Seed for the random number generator.
        Use these keys exactly. If not set, will use the global ones which were passed on the object initialization
        :param temp_folder: Path to folder with enough space to hold the memory map of self.d_graph (for big graphs); to be passed joblib.Parallel.temp_folder
        """

        self.graph = graph
        self.dimensions = dimensions
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.p = p
        self.q = q
        self.weight_key = weight_key
        self.workers = workers
        self.quiet = quiet
        self.d_graph = defaultdict(dict)

        if sampling_strategy is None:
            self.sampling_strategy = {}
        else:
            self.sampling_strategy = sampling_strategy

        self.temp_folder, self.require = None, None
        if temp_folder:
            if not os.path.isdir(temp_folder):
                raise NotADirectoryError("temp_folder does not exist or is not a directory. ({})".format(temp_folder))

            self.temp_folder = temp_folder
            self.require = "sharedmem"

        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self._precompute_probabilities()
        self.walks = self._generate_walks()

    def _precompute_probabilities(self):
        """
        Precomputes transition probabilities for each node.
        """

        d_graph = self.d_graph

        nodes_generator = self.graph.nodes() if self.quiet \
            else tqdm(self.graph.nodes(), desc='Computing transition probabilities')

        for source in nodes_generator:

            # Init probabilities dict for first travel
            if self.PROBABILITIES_KEY not in d_graph[source]:
                d_graph[source][self.PROBABILITIES_KEY] = dict()

            for current_node in self.graph.neighbors(source):

                # Init probabilities dict
                if self.PROBABILITIES_KEY not in d_graph[current_node]:
                    d_graph[current_node][self.PROBABILITIES_KEY] = dict()

                unnormalized_weights = list()
                d_neighbors = list()

                # Calculate unnormalized weights
                for destination in self.graph.neighbors(current_node):

                    p = self.sampling_strategy[current_node].get(self.P_KEY,
                                                                 self.p) if current_node in self.sampling_strategy else self.p
                    q = self.sampling_strategy[current_node].get(self.Q_KEY,
                                                                 self.q) if current_node in self.sampling_strategy else self.q

                    try:
                        if self.graph[current_node][destination].get(self.weight_key):
                            weight = self.graph[current_node][destination].get(self.weight_key, 1)
                        else:
                            ## Example : AtlasView({0: {'type': 1, 'weight':0.1}})- when we have edge weight
                            edge = list(self.graph[current_node][destination])[-1]
                            weight = self.graph[current_node][destination][edge].get(self.weight_key, 1)

                    except:
                        weight = 1

                    if destination == source:  # Backwards probability
                        ss_weight = weight * 1 / p
                    elif destination in self.graph[source]:  # If the neighbor is connected to the source
                        ss_weight = weight
                    else:
                        ss_weight = weight * 1 / q

                    # Assign the unnormalized sampling strategy weight, normalize during random walk
                    unnormalized_weights.append(ss_weight)
                    d_neighbors.append(destination)

                # Normalize
                unnormalized_weights = np.array(unnormalized_weights)
                d_graph[current_node][self.PROBABILITIES_KEY][
                    source] = unnormalized_weights / unnormalized_weights.sum()

            # Calculate first_travel weights for source
            first_travel_weights = []

            for destination in self.graph.neighbors(source):
                first_travel_weights.append(self.graph[source][destination].get(self.weight_key, 1))

            first_travel_weights = np.array(first_travel_weights)
            d_graph[source][self.FIRST_TRAVEL_KEY] = first_travel_weights / first_travel_weights.sum()

            # Save neighbors
            d_graph[source][self.NEIGHBORS_KEY] = list(self.graph.neighbors(source))

    def _generate_walks(self) -> list:
        """
        Generates the random walks which will be used as the skip-gram input.
        :return: List of walks. Each walk is a list of nodes.
        """

        flatten = lambda l: [item for sublist in l for item in sublist]

        # Split num_walks for each worker
        num_walks_lists = np.array_split(range(self.num_walks), self.workers)

        walk_results = Parallel(n_jobs=self.workers, temp_folder=self.temp_folder, require=self.require)(
            delayed(parallel_generate_walks)(self.d_graph,
                                             self.walk_length,
                                             len(num_walks),
                                             idx,
                                             self.sampling_strategy,
                                             self.NUM_WALKS_KEY,
                                             self.WALK_LENGTH_KEY,
                                             self.NEIGHBORS_KEY,
                                             self.PROBABILITIES_KEY,
                                             self.FIRST_TRAVEL_KEY,
                                             self.quiet) for
            idx, num_walks
            in enumerate(num_walks_lists, 1))

        walks = flatten(walk_results)

        return walks

    def fit(self, **skip_gram_params) -> gensim.models.Word2Vec:
        """
        Creates the embeddings using gensim's Word2Vec.
        :param skip_gram_params: Parameters for gensim.models.Word2Vec - do not supply 'size' / 'vector_size' it is
            taken from the Node2Vec 'dimensions' parameter
        :type skip_gram_params: dict
        :return: A gensim word2vec model
        """

        if 'workers' not in skip_gram_params:
            skip_gram_params['workers'] = self.workers

        # Figure out gensim version, naming of output dimensions changed from size to vector_size in v4.0.0
        gensim_version = pkg_resources.get_distribution("gensim").version
        size = 'size' if gensim_version < '4.0.0' else 'vector_size'
        if size not in skip_gram_params:
            skip_gram_params[size] = self.dimensions

        if 'sg' not in skip_gram_params:
            skip_gram_params['sg'] = 1

        return gensim.models.Word2Vec(self.walks, **skip_gram_params)


In [3]:
"""
data collect
"""
twitter_bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOYLagEAAAAA1K8YrEuA8CHQDAqAdjkPsBS2Pig%3DMUmnQgjpzkkXslyJpeNytAwFQ2qgiGE0Ah0rkrjuwH9UnOYSLI'
dataset = MuminDataset(twitter_bearer_token=twitter_bearer_token,size='medium')
dataset.compile()
dataset.add_embeddings()
if 'dgl_graph' not in globals():
    dgl_graph = dataset.to_dgl()


2022-09-16 10:09:08,943 [INFO] Loading dataset
2022-09-16 10:09:46,780 [INFO] Outputting to DGL


In [4]:
"""
graph1
"""
rel = ('tweet', 'posted_inv', 'user')
g1 = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
num = g1.edges()[0].shape[0]
g_src = g1.edges()[0].tolist()
g_tgt = g1.edges()[1].tolist()
src = []
tgt = []
for i in range(num):
    for j in range(i,num):
        if g_tgt[i] == g_tgt[j]:
            src.append(g_src[i])
            tgt.append(g_src[j])
graph = nx.Graph()
for i in range(len(src)):
    graph.add_edge(src[i],tgt[i])

tgt_node = 'tweet'
dim = g1.nodes[tgt_node].data['feat'].shape[1]

In [5]:
# Precompute probabilities and generate walks

node2vec = Node2Vec(graph, dimensions=dim, walk_length=20, num_walks=100, workers=4)

## if d_graph is big enough to fit in the memory, pass temp_folder which has enough disk space
# Note: It will trigger "sharedmem" in Parallel, which will be slow on smaller graphs
#node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")

Computing transition probabilities: 100%|██████████| 10475/10475 [01:25<00:00, 122.89it/s]


In [6]:
# Embed
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

2022-09-16 10:12:43,867 [INFO] collecting all words and their counts
2022-09-16 10:12:43,867 [INFO] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-09-16 10:12:43,891 [INFO] PROGRESS: at sentence #10000, processed 200000 words, keeping 10229 word types
2022-09-16 10:12:43,914 [INFO] PROGRESS: at sentence #20000, processed 400000 words, keeping 10475 word types
2022-09-16 10:12:43,935 [INFO] PROGRESS: at sentence #30000, processed 600000 words, keeping 10475 word types
2022-09-16 10:12:43,957 [INFO] PROGRESS: at sentence #40000, processed 800000 words, keeping 10475 word types
2022-09-16 10:12:43,978 [INFO] PROGRESS: at sentence #50000, processed 1000000 words, keeping 10475 word types
2022-09-16 10:12:44,000 [INFO] PROGRESS: at sentence #60000, processed 1200000 words, keeping 10475 word types
2022-09-16 10:12:44,021 [INFO] PROGRESS: at sentence #70000, processed 1400000 words, keeping 10475 word types
2022-09-16 10:12:44,043 [INFO] PROGRESS: at sentence #80000, 

In [7]:
nodes = list(graph.nodes())
emb = []
for i in nodes:
    emb.append(model.wv[str(i)])

import pandas as pd
dataframe = pd.DataFrame(nodes,columns=['node'])
dataframe['feats'] = emb

sub_g = g1
feats = sub_g.nodes[tgt_node].data['feat'].cpu().detach().numpy()
text_feats= []
for i in range(feats.shape[0]):
    text_feats.append(feats[i,:])

label = sub_g.nodes[tgt_node].data['label'].cpu().detach().numpy()
train_mask = sub_g.nodes[tgt_node].data['train_mask'].cpu().detach().numpy()
val_mask = sub_g.nodes[tgt_node].data['val_mask'].cpu().detach().numpy()
test_mask = sub_g.nodes[tgt_node].data['test_mask'].cpu().detach().numpy()

import pandas as pd
dataframe['label'] = label.tolist()
dataframe['train_mask'] = train_mask.tolist()
dataframe['val_mask'] = val_mask.tolist()
dataframe['test_mask'] = test_mask.tolist()
dataframe['text_feats'] = text_feats

dc = nx.degree_centrality(graph)
bc = nx.betweenness_centrality(graph)
dc_list = []
for key in sorted(dc.keys()):
    dc_list.append(dc[key])

bc_list = []
for key in sorted(bc.keys()):
    bc_list.append(bc[key])

dc_feats = []
for i in range(len(dataframe)):
    dc_feats.append((dc_list[i]*dataframe['feats'][i]+(1-dc_list[i])*dataframe['text_feats'][i]))

bc_feats = []
for i in range(len(dataframe)):
    bc_feats.append((bc_list[i]*dataframe['feats'][i]+(1-bc_list[i])*dataframe['text_feats'][i]))

dataframe['dc_feats'] = dc_feats
dataframe['bc_feats'] = bc_feats

mix_feats = []
for i in range(len(dataframe)):
    mix_feats.append(np.hstack((dataframe['feats'][i],dataframe['text_feats'][i])))
dataframe['mix_feats'] = mix_feats

# Split up the data
train = dataframe.query('train_mask == True')
val = dataframe.query('val_mask == True')
test = dataframe.query('test_mask == True')

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [8]:
def getFeatsMatrix(df):
    m_temp = np.zeros([len(df['feats'].values),dim])
    for i in range(len(df['feats'].values)):
        m_temp[i,:] = df['feats'][i]
    return m_temp

def getTextFeatsMatrix(df):
    m_temp = np.zeros([len(df['text_feats'].values),dim])
    for i in range(len(df['text_feats'].values)):
        m_temp[i,:] = df['text_feats'][i]
    return m_temp

def getMixFeatsMatrix(df):
    m_temp = np.zeros([len(df['mix_feats'].values),2*dim])
    for i in range(len(df['mix_feats'].values)):
        m_temp[i,:] = df['mix_feats'][i]
    return m_temp

def getDcFeatsMatrix(df):
    m_temp = np.zeros([len(df['dc_feats'].values),dim])
    for i in range(len(df['dc_feats'].values)):
        m_temp[i,:] = df['dc_feats'][i]
    return m_temp

def getBcFeatsMatrix(df):
    m_temp = np.zeros([len(df['bc_feats'].values),dim])
    for i in range(len(df['bc_feats'].values)):
        m_temp[i,:] = df['bc_feats'][i]
    return m_temp

In [9]:
from torch.utils.data import DataLoader, TensorDataset
def convert_to_data_loader(dataset,m, num_classes):
    # convert from list to tensor
    input_tensor = torch.from_numpy(m).float()
    label_tensor = torch.from_numpy(np.array(dataset['label'])).long()
    tensor_dataset = TensorDataset(input_tensor, label_tensor)
    loader = DataLoader(tensor_dataset, batch_size=64, shuffle=True)

    return loader

num_classes = 2   # number of possible labels in the sentiment analysis task


# train_loader = convert_to_data_loader(train,getMixFeatsMatrix(train), num_classes)
# dev_loader = convert_to_data_loader(val, getMixFeatsMatrix(val),num_classes)
# test_loader = convert_to_data_loader(test,getMixFeatsMatrix(test), num_classes)


In [10]:
from torch import nn, optim
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)


        # 构造Dropout方法，在每次训练过程中都随机“掐死”百分之二十的神经元，防止过拟合。
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # 确保输入的tensor是展开的单列数据，把每张图片的通道、长度、宽度三个维度都压缩为一列
        x = x.view(x.shape[0], -1)

        # 在训练过程中对隐含层神经元的正向推断使用Dropout方法
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # 在输出单元不需要使用Dropout方法
        x = F.log_softmax(self.fc4(x), dim=1)

        return x

In [11]:
def train_nn():
    # 对上面定义的Classifier类进行实例化
    model = Classifier()

    # 定义损失函数为负对数损失函数
    criterion = nn.NLLLoss()

    # 优化方法为Adam梯度下降方法，学习率为0.003
    optimizer = optim.Adam(model.parameters(), lr=0.003)

    # 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
    epochs = 20

    # 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
    train_losses, test_losses = [], []

    for e in range(epochs):
        running_loss = 0

        # 对训练集中的所有图片都过一遍
        for images, labels in train_loader:
            # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
            optimizer.zero_grad()

            # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 每次学完一遍数据集，都进行以下测试操作
        else:
            test_loss = 0
            accuracy = 0
            # 测试的时候不需要开自动求导和反向传播
            with torch.no_grad():
                # 关闭Dropout
                model.eval()

                # 对测试集中的所有图片都过一遍
                for images, labels in dev_loader:
                    # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
                    log_ps = model(images)
                    test_loss += criterion(log_ps, labels)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)

                    # 等号右边为每一批64张测试图片中预测正确的占比
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            # 恢复Dropout
            model.train()
            # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(dev_loader))

    return model

In [12]:
def predict_nn(trained_model, test_loader):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
    model.eval()

    correct = 0  # count the number of correct classification labels

    gold_labs = []  # gold labels to return
    pred_labs = []  # predicted labels to return

    for inputs, labels in test_loader:
        test_output = trained_model(inputs)
        predicted_labels = test_output.argmax(1)

        gold_labs.extend(labels.tolist())
        pred_labs.extend(predicted_labels.tolist())


    f1 = f1_score(gold_labs, pred_labs, average='macro')

    return f1

In [13]:
train_loader = convert_to_data_loader(train,getBcFeatsMatrix(train), num_classes)
dev_loader = convert_to_data_loader(val, getBcFeatsMatrix(val),num_classes)
test_loader = convert_to_data_loader(test,getBcFeatsMatrix(test), num_classes)

model = train_nn()
score=predict_nn(model,test_loader)

print(f'F1 score (macro average) = {score}')


F1 score (macro average) = 0.48561005269558166


In [14]:
train_loader = convert_to_data_loader(train,getDcFeatsMatrix(train), num_classes)
dev_loader = convert_to_data_loader(val, getDcFeatsMatrix(val),num_classes)
test_loader = convert_to_data_loader(test,getDcFeatsMatrix(test), num_classes)



model = train_nn()
score=predict_nn(model,test_loader)

print(f'F1 score (macro average) = {score}')


F1 score (macro average) = 0.48561005269558166


In [15]:
train_loader = convert_to_data_loader(train,getFeatsMatrix(train), num_classes)
dev_loader = convert_to_data_loader(val, getFeatsMatrix(val),num_classes)
test_loader = convert_to_data_loader(test,getFeatsMatrix(test), num_classes)
model = train_nn()
score=predict_nn(model,test_loader)

print(f'F1 score (macro average) = {score}')


F1 score (macro average) = 0.5914227760069545


In [16]:
train_loader = convert_to_data_loader(train,getMixFeatsMatrix(train), num_classes)
dev_loader = convert_to_data_loader(val, getMixFeatsMatrix(val),num_classes)
test_loader = convert_to_data_loader(test,getMixFeatsMatrix(test), num_classes)

from torch import nn, optim
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2*dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)


        # 构造Dropout方法，在每次训练过程中都随机“掐死”百分之二十的神经元，防止过拟合。
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # 确保输入的tensor是展开的单列数据，把每张图片的通道、长度、宽度三个维度都压缩为一列
        x = x.view(x.shape[0], -1)

        # 在训练过程中对隐含层神经元的正向推断使用Dropout方法
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # 在输出单元不需要使用Dropout方法
        x = F.log_softmax(self.fc4(x), dim=1)

        return x

model = train_nn()
score=predict_nn(model,test_loader)

print(f'F1 score (macro average) = {score}')


F1 score (macro average) = 0.5946204957832865
